In [39]:
!python -V

Python 3.9.21


In [10]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==1.6.1

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
  Using cached scikit_learn-1.6.1-cp39-cp39-macosx_10_9_x86_64.whl.metadata (31 kB)
Using cached scikit_learn-1.6.1-cp39-cp39-macosx_10_9_x86_64.whl (12.1 MB)


In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import root_mean_squared_error

In [5]:
import sklearn
import sys
print("sklearn version:", sklearn.__version__)
print("python executable:", sys.executable)

sklearn version: 1.6.1
python executable: /Users/zihaozhou/.conda/envs/exp-tracking-env/bin/python


In [6]:
from sklearn.metrics import mean_squared_error
import inspect

print(mean_squared_error)
print(inspect.getfile(mean_squared_error))


<function mean_squared_error at 0x14c8c7c10>
/Users/zihaozhou/.conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/utils/_param_validation.py


In [1]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/zihaozhou/VS_code/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1743464550778, experiment_id='1', last_update_time=1743464550778, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [2]:
print(mlflow.__version__)

2.21.3


In [8]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [21]:
df_train = read_dataframe('data/green_tripdata_2023-01.parquet')
df_val = read_dataframe('data/green_tripdata_2023-02.parquet')

In [22]:
len(df_train), len(df_val)

(65946, 62574)

In [23]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [24]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [25]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [27]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mse = mean_squared_error(y_val, y_pred)**0.5
mse

6.037585607307625

In [28]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [30]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "data/green_tripdata_2023-01.parquet")
    mlflow.log_param("valid-data-path", "data/green_tripdata_2023-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred)**0.5
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [31]:
import xgboost as xgb

In [33]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [32]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [37]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred)**0.5
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [38]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:7.78817                           
[1]	validation-rmse:6.81558                           
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/zihaozhou/.conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [21:22:14] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[2]	validation-rmse:6.22256                           
[3]	validation-rmse:5.86902                           
[4]	validation-rmse:5.66049                           
[5]	validation-rmse:5.54098                           
[6]	validation-rmse:5.46869                           
[7]	validation-rmse:5.42044                           
[8]	validation-rmse:5.39151                           
[9]	validation-rmse:5.36926                           
[10]	validation-rmse:5.35458                          
[11]	validation-rmse:5.34420                          
[12]	validation-rmse:5.33642                          
[13]	validation-rmse:5.32968                          
[14]	validation-rmse:5.32611                          
[15]	validation-rmse:5.32057                          
[16]	validation-rmse:5.31684                          
[17]	validation-rmse:5.31311                          
[18]	validation-rmse:5.30948                          
[19]	validation-rmse:5.30819                          
[20]	valid

/Users/zihaozhou/.conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [21:22:29] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:5.85340                                                    
[1]	validation-rmse:5.45193                                                    
[2]	validation-rmse:5.39195                                                    
[3]	validation-rmse:5.38008                                                    
[4]	validation-rmse:5.36756                                                    
[5]	validation-rmse:5.36670                                                    
[6]	validation-rmse:5.36663                                                    
[7]	validation-rmse:5.35902                                                    
[8]	validation-rmse:5.35670                                                    
[9]	validation-rmse:5.35448                                                    
[10]	validation-rmse:5.34931                                                   
[11]	validation-rmse:5.34756                                                   
[12]	validation-rmse:5.33260            

...

[1]	validation-rmse:7.95272                                                    
[2]	validation-rmse:7.43779                                                    
[3]	validation-rmse:7.01481                                                    
[4]	validation-rmse:6.67103                                                    
[5]	validation-rmse:6.39203                                                    
[6]	validation-rmse:6.16629                                                    
[7]	validation-rmse:5.98500                                                    
[8]	validation-rmse:5.83836                                                    
[9]	validation-rmse:5.72225                                                    
[10]	validation-rmse:5.62765                                                   
[11]	validation-rmse:5.55328                                                   
[12]	validation-rmse:5.49443                                                   
[13]	validation-rmse:5.44658            

/Users/zihaozhou/.conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [21:39:58] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:7.53923                                                    
[2]	validation-rmse:6.94862                                                    
[3]	validation-rmse:6.50632                                                    
[4]	validation-rmse:6.17614                                                    
[5]	validation-rmse:5.93231                                                    
[6]	validation-rmse:5.75273                                                    
[7]	validation-rmse:5.62167                                                    
[8]	validation-rmse:5.52731                                                    
[9]	validation-rmse:5.45591                                                    
[10]	validation-rmse:5.40296                                                   
[11]	validation-rmse:5.36007                                                   
[12]	validation-rmse:5.33040                                                   
[13]	validation-rmse:5.30789            

/Users/zihaozhou/.conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [21:40:11] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[2]	validation-rmse:5.85879                                                    
[3]	validation-rmse:5.60857                                                    
[4]	validation-rmse:5.47860                                                    
[5]	validation-rmse:5.40802                                                    
[6]	validation-rmse:5.37134                                                    
[7]	validation-rmse:5.34965                                                    
[8]	validation-rmse:5.33747                                                    
[9]	validation-rmse:5.32974                                                    
[10]	validation-rmse:5.32394                                                   
[11]	validation-rmse:5.32005                                                   
[12]	validation-rmse:5.31743                                                   
[13]	validation-rmse:5.31531                                                   
[14]	validation-rmse:5.31330            

In [47]:
mlflow.xgboost.autolog(disable=True)

In [53]:
with mlflow.start_run(nested=True):
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params= {
       "learning_rate":0.0923004677969142,
       "max_depth":43,
       "min_child_weight":3.158825903573778,
       "objective": "reg:linear",
       "reg_alpha": 0.020676403448522406,
       "reg_lambda": 0.2007432700080768,
       "seed": 42
    }

    # mlflow.xgboost.autolog()
    
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred)**0.5
    mlflow.log_metric("rmse", rmse)


with open("models/preprocessor.b", "wb") as f_out:
    pickle.dump(dv, f_out)  # store the dv for future reuse
    
mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")



# mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/Users/zihaozhou/.conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:29:26] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:8.75190
[1]	validation-rmse:8.25251
[2]	validation-rmse:7.81542
[3]	validation-rmse:7.43602
[4]	validation-rmse:7.10675
[5]	validation-rmse:6.82331
[6]	validation-rmse:6.57866
[7]	validation-rmse:6.37034
[8]	validation-rmse:6.19239
[9]	validation-rmse:6.04067
[10]	validation-rmse:5.91083
[11]	validation-rmse:5.80076
[12]	validation-rmse:5.70727
[13]	validation-rmse:5.62761
[14]	validation-rmse:5.56129
[15]	validation-rmse:5.50389
[16]	validation-rmse:5.45596
[17]	validation-rmse:5.41620
[18]	validation-rmse:5.38238
[19]	validation-rmse:5.35287
[20]	validation-rmse:5.32736
[21]	validation-rmse:5.30604
[22]	validation-rmse:5.28679
[23]	validation-rmse:5.27148
[24]	validation-rmse:5.25761
[25]	validation-rmse:5.24528
[26]	validation-rmse:5.23584
[27]	validation-rmse:5.22704
[28]	validation-rmse:5.22049
[29]	validation-rmse:5.21424
[30]	validation-rmse:5.20864
[31]	validation-rmse:5.20435
[32]	validation-rmse:5.20088
[33]	validation-rmse:5.19687
[34]	validation-rmse:5.1

2025/04/01 22:29:54 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/zihaozhou/.conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:29:54] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."


TypeError: cannot pickle 'module' object

In [70]:
with mlflow.start_run():
    

   best_params= {"objective": 'reg:squarederror',
                 "n_estimators" :100,
                 "learning_rate":0.0923004677969142,
                 "max_depth":40,
                 "min_child_weight":3.158825903573778,
                 "objective": "reg:linear",
                 "reg_alpha": 0.020676403448522406,
                 "reg_lambda": 0.2007432700080768,
                 "seed": 42
                 }


   xgb_regressor = xgb.XGBRegressor(**best_params)

   # mlflow.xgboost.autolog()
   
   xgb_regressor.fit(X_train, y_train)
   y_pred = xgb_regressor.predict(X_val)

   # Evaluate the model

   rmse = mean_squared_error(y_val, y_pred)**0.5
   mlflow.log_metric("rmse", rmse)
   with open("models/xgb.bin", "wb") as f_out:
      pickle.dump((dv,xgb_regressor), f_out)  

mlflow.log_artifact("models/xgb.bin", artifact_path="models_pickle")


/Users/zihaozhou/.conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [23:45:56] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
2025/04/01 23:46:03 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/zihaozhou/.conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [23:46:03] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."


In [59]:
#load model from saved artifact 
logged_model ='runs:/c5fd346b28394d5faf39d881a163b7b4/models_mlflow'
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [60]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: c5fd346b28394d5faf39d881a163b7b4

In [72]:
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model

In [88]:
y_pred= xgboost_model.predict(valid)



In [89]:
y_pred[:3]

array([22.335491, 20.831606, 24.71197 ], dtype=float32)

## Options 2 

In [ ]:
#if we use artifact instead
artifact_path = "models_pickle/xgb.bin"
run_id = "50215fbe793a49f7bc383d7c5a347586"
local_path = mlflow.artifacts.download_artifacts(
    run_id=run_id,
    artifact_path=artifact_path
    )

In [80]:
# Load with XGBoost
with open(local_path, "rb") as f:
    model = pickle.load(f)

In [85]:
dv, xgb_model = model 

In [91]:

y_pred_bin= xgb_model.predict(X_val)

In [92]:
y_pred_bin[:3]

array([21.646782, 21.360643, 24.527328], dtype=float32)

In [79]:
import os
print("File path:", local_path)
print("File size:", os.path.getsize(local_path), "bytes")

File path: /Users/zihaozhou/VS_code/mlops-zoomcamp/02-experiment-tracking/mlruns/1/50215fbe793a49f7bc383d7c5a347586/artifacts/models_pickle/xgb.bin
File size: 4387188 bytes


##  Log serveral models  for  prepairation of model registry 

In [153]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()
dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "data/green_tripdata_2023-01.parquet")
        mlflow.log_param("valid-data-path", "data/green_tripdata_2023-02.parquet")
    

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred)**0.5
        mlflow.log_metric("rmse", rmse)
        with open("models/preprocessor.b", "wb") as f_out:
            pickle.dump((dv,mlmodel), f_out)  

        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2025/04/03 20:07:21 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.21.3, required: mlflow==2.21.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/04/03 20:07:33 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.21.3, required: mlflow==2.21.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/04/03 20:13:59 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.21.3, required: mlflow==2.21.2)
To f

In [97]:
from mlflow.tracking import MlflowClient
mlflow_tracking_uri = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=mlflow_tracking_uri)

In [100]:
experiments = client.search_experiments()
for exp in experiments:
    print (f'Nane: {exp.name} , ID:{exp.experiment_id}, artifact location :{exp.artifact_location}')

Nane: nyc-taxi-experiment , ID:1, artifact location :/Users/zihaozhou/VS_code/mlops-zoomcamp/02-experiment-tracking/mlruns/1
Nane: Default , ID:0, artifact location :mlflow-artifacts:/0


In [105]:
from mlflow.entities import ViewType
runs = client.search_runs(experiment_ids='1',
                          filter_string='',
                          run_view_type= ViewType.ACTIVE_ONLY,
                          max_results=5,
                          order_by=['metrics.rmse ASC']
                          )
for run in runs:
    print(f"run id: {run.info.run_id}, rmse : {run.data.metrics['rmse']:.4f}")

run id: 23b3fda0ae8a483c9cd0f59389d7a4e4, rmse : 5.1147
run id: f10256a9713c443e977523ac4a695a1d, rmse : 5.1147
run id: 1cbcfc48cbd24c7ab4afab6656c2c242, rmse : 5.1147
run id: f9610d3a40c64bdfadaab62e54ba8eb0, rmse : 5.1147
run id: 9388a15b64f443c695cd88f8fb136afa, rmse : 5.1147


In [107]:
from mlflow.entities import ViewType
runs = client.search_runs(experiment_ids='1',
                          filter_string='metric.rmse < 5.12',
                          run_view_type= ViewType.ACTIVE_ONLY,
                          max_results=10,
                          order_by=['metrics.rmse ASC']
                          )
for run in runs:
    print(f"run id: {run.info.run_id}, rmse : {run.data.metrics['rmse']:.4f}")

run id: 23b3fda0ae8a483c9cd0f59389d7a4e4, rmse : 5.1147
run id: f10256a9713c443e977523ac4a695a1d, rmse : 5.1147
run id: 1cbcfc48cbd24c7ab4afab6656c2c242, rmse : 5.1147
run id: f9610d3a40c64bdfadaab62e54ba8eb0, rmse : 5.1147
run id: 9388a15b64f443c695cd88f8fb136afa, rmse : 5.1147
run id: 97fbf6c3cba14d4e9a64df93b2c47593, rmse : 5.1157


In [122]:
# next to find models hving lowest rmse and duration 
run_infos = []
runs = client.search_runs(experiment_ids='1',
                          filter_string='',
                          run_view_type= ViewType.ACTIVE_ONLY,
                          max_results=10,
                          order_by=['metrics.rmse ASC']
                          )
for run in runs:
    rmse = run.data.metrics.get("rmse")
    start = run.info.start_time
    end = run.info.end_time
    if rmse is not None and start is not None and end is not None:
        duration = (end - start) / 1000  # Convert ms to seconds
        run_infos.append({
            "run_id": run.info.run_id,
            "rmse": rmse,
            "duration_sec": duration
        })

run_infos

[{'run_id': '23b3fda0ae8a483c9cd0f59389d7a4e4',
  'rmse': 5.11472819026649,
  'duration_sec': 33.625},
 {'run_id': 'f10256a9713c443e977523ac4a695a1d',
  'rmse': 5.11472819026649,
  'duration_sec': 33.245},
 {'run_id': '1cbcfc48cbd24c7ab4afab6656c2c242',
  'rmse': 5.11472819026649,
  'duration_sec': 43.873},
 {'run_id': 'f9610d3a40c64bdfadaab62e54ba8eb0',
  'rmse': 5.11472819026649,
  'duration_sec': 36.964},
 {'run_id': '9388a15b64f443c695cd88f8fb136afa',
  'rmse': 5.11472819026649,
  'duration_sec': 262.577},
 {'run_id': '97fbf6c3cba14d4e9a64df93b2c47593',
  'rmse': 5.115681014129676,
  'duration_sec': 18.229},
 {'run_id': '4b784513b8e746489ae29d9e632dd551',
  'rmse': 5.12187644499111,
  'duration_sec': 13.135},
 {'run_id': 'd0a87225223f4dc88837b3c9e74289b7',
  'rmse': 5.126511500125477,
  'duration_sec': 26.849},
 {'run_id': 'f39a48180fa74a54a0652fe896686c32',
  'rmse': 5.128890829962695,
  'duration_sec': 25.31},
 {'run_id': 'b9c4d8c6330a4f7aa10338363e9fd7de',
  'rmse': 5.1293643771

In [125]:
#sort 
sorted_runs = sorted(
    run_infos,
    key=lambda x: (x["rmse"], x["duration_sec"]))[:3]  # First by rmse, then duration


In [126]:
sorted_runs 

[{'run_id': 'f10256a9713c443e977523ac4a695a1d',
  'rmse': 5.11472819026649,
  'duration_sec': 33.245},
 {'run_id': '23b3fda0ae8a483c9cd0f59389d7a4e4',
  'rmse': 5.11472819026649,
  'duration_sec': 33.625},
 {'run_id': 'f9610d3a40c64bdfadaab62e54ba8eb0',
  'rmse': 5.11472819026649,
  'duration_sec': 36.964}]

In [130]:
# I would register the 3 models with least rsme and duration 
for i in sorted_runs:
    model_uri = f"runs:/{i.get('run_id')}/model"
    mlflow.register_model(model_uri=model_uri,)name='nyc-taxi-regiessor'

Registered model 'nyc-taxi-regiessor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regiessor'.
Registered model 'nyc-taxi-regiessor' already exists. Creating a new version of this model...
Created version '4' of model 'nyc-taxi-regiessor'.
Registered model 'nyc-taxi-regiessor' already exists. Creating a new version of this model...
Created version '5' of model 'nyc-taxi-regiessor'.


In [137]:
# let check what models we have under registry
all_versions = client.search_model_versions(filter_string= "name='nyc-taxi-regiessor'")

for v in all_versions:
    print(f"Version: {v.version}, Stage: {v.current_stage}, Status: {v.status}, Aliases: {v.aliases}")

Version: 5, Stage: None, Status: READY, Aliases: []
Version: 4, Stage: None, Status: READY, Aliases: []
Version: 3, Stage: None, Status: READY, Aliases: []
Version: 2, Stage: None, Status: READY, Aliases: []
Version: 1, Stage: None, Status: READY, Aliases: []


In [142]:
client.transition_model_version_stage(name= 'nyc-taxi-regiessor',
                                      version=3,
                                      stage='production',
                                      archive_existing_versions=False)


/var/folders/xv/p1z69rt949l1ttxwv1f6vr340000gn/T/ipykernel_39856/1600414815.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(name= 'nyc-taxi-regiessor',


<ModelVersion: aliases=[], creation_timestamp=1743718988470, current_stage='Production', description=None, last_updated_timestamp=1743721981238, name='nyc-taxi-regiessor', run_id='f10256a9713c443e977523ac4a695a1d', run_link=None, source='/Users/zihaozhou/VS_code/mlops-zoomcamp/02-experiment-tracking/mlruns/1/f10256a9713c443e977523ac4a695a1d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [143]:
client.transition_model_version_stage(name= 'nyc-taxi-regiessor',
                                      version=4,
                                      stage='staging',
                                      archive_existing_versions=False)

/var/folders/xv/p1z69rt949l1ttxwv1f6vr340000gn/T/ipykernel_39856/2275782555.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(name= 'nyc-taxi-regiessor',


<ModelVersion: aliases=[], creation_timestamp=1743718988487, current_stage='Staging', description=None, last_updated_timestamp=1743722134451, name='nyc-taxi-regiessor', run_id='23b3fda0ae8a483c9cd0f59389d7a4e4', run_link=None, source='/Users/zihaozhou/VS_code/mlops-zoomcamp/02-experiment-tracking/mlruns/1/23b3fda0ae8a483c9cd0f59389d7a4e4/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [144]:
# we can also update the description 
client.update_registered_model(name='nyc-taxi-regiessor',
                               description='This is the registry for ny taxi fare project'
                               )

<RegisteredModel: aliases={'challenger': 1, 'champion': 2}, creation_timestamp=1743650476176, description='This is the registry for ny taxi fare project', last_updated_timestamp=1743722673960, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1743718988498, current_stage='None', description=None, last_updated_timestamp=1743718988498, name='nyc-taxi-regiessor', run_id='f9610d3a40c64bdfadaab62e54ba8eb0', run_link=None, source='/Users/zihaozhou/VS_code/mlops-zoomcamp/02-experiment-tracking/mlruns/1/f9610d3a40c64bdfadaab62e54ba8eb0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>,
 <ModelVersion: aliases=[], creation_timestamp=1743718988470, current_stage='Production', description=None, last_updated_timestamp=1743721981238, name='nyc-taxi-regiessor', run_id='f10256a9713c443e977523ac4a695a1d', run_link=None, source='/Users/zihaozhou/VS_code/mlops-zoomcamp/02-experiment-tracking/mlruns/1/f10256a9713c443e977523ac4a695a1d/artifacts/model', s

In [145]:
#we can also update the description for each version of model 
client.update_model_version(name='nyc-taxi-regiessor',
                            version= 3,
                            description='The model is transit to production'
                            )

<ModelVersion: aliases=[], creation_timestamp=1743718988470, current_stage='Production', description='The model is transit to production', last_updated_timestamp=1743722789499, name='nyc-taxi-regiessor', run_id='f10256a9713c443e977523ac4a695a1d', run_link=None, source='/Users/zihaozhou/VS_code/mlops-zoomcamp/02-experiment-tracking/mlruns/1/f10256a9713c443e977523ac4a695a1d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [147]:
from sklearn.metrics import root_mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [148]:
df = read_dataframe('data/green_tripdata_2023-03.parquet')

In [154]:
with open('models/preprocessor.b','rb') as f:
    dv,model = pickle.load(f)

In [155]:
X_test = preprocess(df,dv)

In [156]:
target = "duration"
y_test = df[target].values

In [157]:
%time test_model(name='nyc-taxi-regiessor', stage="Production", X_test=X_test, y_test=y_test)

2025/04/03 20:19:00 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.21.3, required: mlflow==2.21.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


CPU times: user 16.5 s, sys: 391 ms, total: 16.9 s
Wall time: 1.89 s


{'rmse': 5.344813683865913}

In [158]:
%time test_model(name='nyc-taxi-regiessor', stage="Staging", X_test=X_test, y_test=y_test)

2025/04/03 20:19:17 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.21.3, required: mlflow==2.21.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


CPU times: user 15.8 s, sys: 292 ms, total: 16.1 s
Wall time: 1.78 s


{'rmse': 5.344813683865913}